In [16]:
!git clone https://github.com/vishalpathak24/img-latex.git

Cloning into 'img-latex'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 1), reused 19 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 8.92 KiB | 8.92 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
!pip install torch

In [ ]:
!pip install 'transformers[torch]'

In [ ]:
!pip install torchserve torch-model-archiver torch-workflow-archiver

In [1]:
%cd img-latex

/home/studio-lab-user/sagemaker-studiolab-notebooks/img-latex/img-latex


In [11]:
!pwd

/home/studio-lab-user/sagemaker-studiolab-notebooks/img-latex/img-latex


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
from transformers import VisionEncoderDecoderModel
from transformers.models.nougat import NougatTokenizerFast
from nougat_latex import NougatLaTexProcessor
import torch

# Model Basic Usage

In [ ]:
model_name = "Norm/nougat-latex-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)

tokenizer = NougatTokenizerFast.from_pretrained(model_name)
latex_processor = NougatLaTexProcessor.from_pretrained(model_name)

In [7]:
from PIL import Image

image = Image.open("../sample-images/lt-2.jpg")
if not image.mode == "RGB":
    image = image.convert('RGB')

In [8]:
pixel_values = latex_processor(image, return_tensors="pt").pixel_values
decoder_input_ids = tokenizer(tokenizer.bos_token, add_special_tokens=False,
                              return_tensors="pt").input_ids

In [9]:
with torch.no_grad():
    outputs = model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=model.decoder.config.max_length,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True,
        num_beams=5,
        bad_words_ids=[[tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

In [10]:
sequence = tokenizer.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(tokenizer.eos_token, "").replace(tokenizer.pad_token, "").replace(tokenizer.bos_token, "")
print(sequence)

\textstyle{\frac{1}{3}}\times\left(6\times{\frac{4}{3}}\right)


# Creating Mar for inferencing

In [13]:
import shutil

shutil.make_archive('nougat', 'zip', 'nougat_latex')

'/home/studio-lab-user/sagemaker-studiolab-notebooks/img-latex/img-latex/nougat.zip'

In [14]:
torch.save(model.state_dict(), 'model.pt')

In [15]:
!torch-model-archiver --model-name img-latex \
--version 1.0 --model-file model.py \
--serialized-file model.pt \
--handler handler.py \
--extra-files "nougat.zip"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
!mkdir ../../model-store/
!mv img-latex.mar ../../model-store/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
!pip install torchserve

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [47]:
!torchserve --model-store model-store/ --models img-latex=img-latex.mar --ts-config img-latex/config.properties &

2024-12-16T12:13:38,056 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager -  Loading snapshot serializer plugin...
nvidia-smi not available or failed: Cannot run program "nvidia-smi": error=2, No such file or directory
2024-12-16T12:13:38,147 [DEBUG] main org.pytorch.serve.util.ConfigManager - xpu-smi not available or failed: Cannot run program "xpu-smi": error=2, No such file or directory
2024-12-16T12:13:38,151 [WARN ] main org.pytorch.serve.util.ConfigManager - Your torchserve instance can access any URL to load models. When deploying to production, make sure to limit the set of allowed_urls in config.properties
2024-12-16T12:13:38,227 [INFO ] main org.pytorch.serve.util.TokenAuthorization - 
######
TorchServe now enforces token authorization by default.
This requires the correct token to be provided when calling an API.
Key file located at /content/img-latex/img-latex/key_file.json
Check token authorization documenation for information: https://github.com/pytorch/serve/

In [20]:
!torchserve --stop

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TorchServe has stopped.


In [49]:
pip install captum

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>